In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pp install transformers

/bin/bash: pp: command not found


In [4]:
df= pd.read_csv('/kaggle/input/tweet-for-airline/Tweets.csv')

In [5]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
target_map = {'positive':1,'negative':0,'neutral':2}
df['target'] = df['airline_sentiment'].map(target_map)

In [7]:
df2 = df[['text','target']]

In [8]:
df2.head()

,text,target
0,@VirginAmerica What @dhepburn said.,2
1,@VirginAmerica plus you've added commercials t...,1
2,@VirginAmerica I didn't today... Must mean I n...,2
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0


In [9]:
df2.columns = ['sentences','labels']

In [10]:
df2.head()

,sentences,labels
0,@VirginAmerica What @dhepburn said.,2
1,@VirginAmerica plus you've added commercials t...,1
2,@VirginAmerica I didn't today... Must mean I n...,2
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0


In [11]:
df2.to_csv('my_data.csv',index =None)

In [12]:
!pip install datasets

In [13]:
from datasets import load_dataset

In [14]:
raw_dataset = load_dataset('csv',data_files = 'my_data.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9bdefcd9f183f916/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 14640
    })
})

In [16]:
raw_dataset['train']

Dataset({
    features: ['sentences', 'labels'],
    num_rows: 14640
})

In [17]:
split = raw_dataset['train'].train_test_split(0.3,seed = 42)

In [18]:
split

DatasetDict({
    train: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 10248
    })
    test: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 4392
    })
})

In [19]:
from transformers import AutoTokenizer

In [20]:
checkpoint = "distilbert-base-cased"

In [21]:
# checkpoint = "distilbert-base-cased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# def tokenize_fn(batch):
#     return tokenizer(batch['sentences'],truncation =True)
# tokenized_datasets = split.map(tokenize_fn,batched  =True)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 3)
# training_args = TrainingArguments(output_dir='training_dir',evaluation_strategy = 'epoch',save_strategy = 'epoch',num_train_epochs = 3,per_device_train_batch_size =16,per_device_eval_batch_size =32)
# trainer.train()

In [22]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [23]:
def tokenize_fn(batch):
    return tokenizer(batch['sentences'],truncation =True)

In [24]:
tokenized_datasets = split.map(tokenize_fn,batched  =True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [25]:
from transformers import AutoModelForSequenceClassification,Trainer,TrainingArguments


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 3)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifi

In [27]:
training_args = TrainingArguments(output_dir='training_dir',evaluation_strategy = 'epoch',save_strategy = 'epoch',num_train_epochs = 3,per_device_train_batch_size =16,per_device_eval_batch_size =32)

In [28]:
def compute_metrics(logits_and_labels):
    logits,labels = logits_and_labels
    predictions = np.argmax(logits,axis=-1)
    acc = np.mean(predictions==labels) 
    return {"accuracy":acc}

In [29]:
trainer = Trainer(model,training_args,train_dataset=tokenized_datasets['train'],eval_dataset = tokenized_datasets['test'],tokenizer = tokenizer,compute_metrics = compute_metrics)

In [30]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentences. If sentences are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10248
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1923
  Number of trainable parameters = 65783811
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.533800,0.415564,0.842213
2,0.336800,0.479513,0.832423
3,0.227300,0.646855,0.834472


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentences. If sentences are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4392
  Batch size = 32
Saving model checkpoint to training_dir/checkpoint-641
Configuration saved in training_dir/checkpoint-641/config.json
Model weights saved in training_dir/checkpoint-641/pytorch_model.bin
tokenizer config file saved in training_dir/checkpoint-641/tokenizer_config.json
Special tokens file saved in training_dir/checkpoint-641/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentences. If sentences are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evalua

TrainOutput(global_step=1923, training_loss=0.3176486471079937, metrics={'train_runtime': 232.5675, 'train_samples_per_second': 132.194, 'train_steps_per_second': 8.269, 'total_flos': 388376501242176.0, 'train_loss': 0.3176486471079937, 'epoch': 3.0})

In [31]:
from transformers import pipeline 
savedmodel = pipeline("text-classification",model = "training_dir/checkpoint-1282",device=0)
split['test']
test_pred = savedmodel(split['test']['sentences'])

loading configuration file training_dir/checkpoint-1282/config.json
Model config DistilBertConfig {
  "_name_or_path": "training_dir/checkpoint-1282",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 28996
}

loading configuration file training_dir/checkpoint-1282/config.json
Model config DistilBertConfig {
  "_

In [32]:
test_pred

[{'label': 'LABEL_1', 'score': 0.9601376056671143},
 {'label': 'LABEL_1', 'score': 0.9126251935958862},
 {'label': 'LABEL_1', 'score': 0.7853383421897888},
 {'label': 'LABEL_1', 'score': 0.6801518201828003},
 {'label': 'LABEL_2', 'score': 0.9523801207542419},
 {'label': 'LABEL_0', 'score': 0.9962283372879028},
 {'label': 'LABEL_2', 'score': 0.8987544178962708},
 {'label': 'LABEL_2', 'score': 0.9667356610298157},
 {'label': 'LABEL_1', 'score': 0.7079578042030334},
 {'label': 'LABEL_0', 'score': 0.9965128302574158},
 {'label': 'LABEL_0', 'score': 0.9680992364883423},
 {'label': 'LABEL_0', 'score': 0.994780957698822},
 {'label': 'LABEL_1', 'score': 0.9454541206359863},
 {'label': 'LABEL_0', 'score': 0.9870538115501404},
 {'label': 'LABEL_2', 'score': 0.9793258905410767},
 {'label': 'LABEL_2', 'score': 0.7897883057594299},
 {'label': 'LABEL_1', 'score': 0.5876857042312622},
 {'label': 'LABEL_0', 'score': 0.9968051910400391},
 {'label': 'LABEL_0', 'score': 0.7976759076118469},
 {'label': 'L

In [33]:
def getlabel(d):
    return int(d['label'].split('_')[1])

In [35]:
output_labels = [getlabel(d) for d in test_pred]

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
print("accuracy is ",accuracy_score(split['test']['labels'],output_labels) )

accuracy is  0.8324225865209471
